# IMPORT

In [16]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import random
import shutil

import joblib
import numpy as np
from easydict import EasyDict as edict
import tensorflow as tf
from PIL import Image, ImageFont, ImageDraw
import cv2
import threading
from six.moves import xrange
import subprocess

# CONFIG (KITTI, SQUEEZENET)

In [15]:
dataset = 'KITTI' #키티 쓸꺼임. 파스칼은 바까볼꺼고
assert dataset.upper()=='PASCAL_VOC' or dataset.upper()=='KITTI', \
      'Currently only support PASCAL_VOC or KITTI dataset'
    
#요즘 대세 컨피그 파일은 tf로 하면 tf에 너무 종속되는느낌이 없잖아 있으니까 이지딕으로 해봄
cfg = edict()

#처음엔 일단 KITTI만. 성공하면 VOC도
cfg.DATASET = dataset.upper()

#cls 설정
if cfg.DATASET == 'PASCAL_VOC':
    # object categories to classify
    cfg.CLASS_NAMES = ('aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
                       'car', 'cat', 'chair', 'cow', 'diningtable', 'dog',
                       'horse', 'motorbike', 'person', 'pottedplant', 'sheep',
                       'sofa', 'train', 'tvmonitor')
elif cfg.DATASET == 'KITTI':
    cfg.CLASS_NAMES = ('car', 'pedestrian', 'cyclist')
    

# number of categories to classify - cls 수
cfg.CLASSES = len(cfg.CLASS_NAMES)    

# ROI pooling output width - 알오아이 풀링 아웃풋 width???  
cfg.GRID_POOL_WIDTH = 7

# ROI pooling output height
cfg.GRID_POOL_HEIGHT = 7

# parameter used in leaky ReLU - 리키렐루용 마이너스쪽 
cfg.LEAKY_COEF = 0.1

# Probability to keep a node in dropout - 드랍아웃 50% 
cfg.KEEP_PROB = 0.5

# image width - 인풋이미지 사이즈 
cfg.IMAGE_WIDTH = 224

# image height
cfg.IMAGE_HEIGHT = 224

# anchor box, array of [cx, cy, w, h]. To be defined later - 앵커박스 아래에 지정해줄건데 센터포인트, 가로세로로 할거임
cfg.ANCHOR_BOX = []

# number of anchor boxes > 앵커수. 9개 보통 함. 
cfg.ANCHORS = len(cfg.ANCHOR_BOX)

# number of anchor boxes per grid #그리드당 앵커 수? 보통 9개 아닌가. 
cfg.ANCHOR_PER_GRID = -1

# batch size #배치사이즈. 여기선 좀 줄여야 할듯
cfg.BATCH_SIZE = 20

# Only keep boxes with probability higher than this threshold #이거 넘는애만 유지하는듯. 앵커인지 프레딕션 뭐 말하는건지는 잘
cfg.PROB_THRESH = 0.005

# Only plot boxes with probability higher than this threshold #플랏용
cfg.PLOT_PROB_THRESH = 0.5

# Bounding boxes with IOU larger than this are going to be removed #아이오유 0.2 이상인애 지워진다??? 보통 아래가 지워지지않나..
cfg.NMS_THRESH = 0.2

# Pixel mean values (BGR order) as a (1, 1, 3) array. Below is the BGR mean #비지알민 -> 부이지지만 쓰는듯 
# of VGG16
cfg.BGR_MEANS = np.array([[[103.939, 116.779, 123.68]]])

# loss coefficient for confidence regression #컨피던스
cfg.LOSS_COEF_CONF = 1.0

# loss coefficient for classification regression #클래시피케이션
cfg.LOSS_COEF_CLASS = 1.0

# loss coefficient for bounding box regression #바운딩박스 레그레션
cfg.LOSS_COEF_BBOX = 10.0

# reduce step size after this many steps #10000스텝마다 스텝사이즈?를 감소함..?
cfg.DECAY_STEPS = 10000

# multiply the learning rate by this factor #러닝레이트 디케이 팩터
cfg.LR_DECAY_FACTOR = 0.1

# learning rate #초기 러닝레이트
cfg.LEARNING_RATE = 0.005

# momentum #옵티마이저 모멘텀 
cfg.MOMENTUM = 0.9

# weight decay #레귤러라이제이션 웨잇디케이
cfg.WEIGHT_DECAY = 0.0005

# wether to load pre-trained model #프리트레인모델 불러오는거.. 스퀴즈넷 또 일시켜야할듯..
cfg.LOAD_PRETRAINED_MODEL = False

# path to load the pre-trained model #프리트레인 모델 패쓰
cfg.PRETRAINED_MODEL_PATH = '만약 내가 프리트레인 모델이 있다면... 없으면 여기서 트레이닝 해야지 뭐... '

# print log to console in debug mode #디버그모드 -> 프린트함수 활성화
cfg.DEBUG_MODE = True

# a small value used to prevent numerical instability -> div 0 제거
cfg.EPSILON = 1e-16

# threshold for safe exponential operation #e? 어디서 쓰이는지 보자..
cfg.EXP_THRESH=1.0

# gradients with norm larger than this is going to be clipped. -> 그래디언트 클리핑
cfg.MAX_GRAD_NORM = 10.0

# Whether to do data augmentation #데이터 오규멘테이션 -> 트레이닝할떄... 보통 디텍션에서는 잘 안쓴다고 들었음. roi도 다 움직이고 귀찮
cfg.DATA_AUGMENTATION = False

# The range to randomly shift the image widht #imdb쪽에서 좀씩 랜덤하게 옮겨서 오규멘테이션 하려고 할떄 쓰는듯
cfg.DRIFT_X = 0

# The range to randomly shift the image height
cfg.DRIFT_Y = 0

# Whether to exclude images harder than hard-category. Only useful for KITTI #하드 이그젬플 막 어클루젼이나 트런캐이티드된거 막 제외하는지
# dataset.
cfg.EXCLUDE_HARD_EXAMPLES = True

# small value used in batch normalization to prevent dividing by 0. The
# default value here is the same with caffe's default value. #카페 디폴트 벨류 맞춤. 배치놈용
cfg.BATCH_NORM_EPSILON = 1e-5

# number of threads to fetch data  #쓰레드 지정 원래 4네 나는 8 해야지
cfg.NUM_THREAD = 8

# capacity for FIFOQueue #큐 캐퍼시티. 보통 배치사이즈 * 3 + @정도를 해주는것으로 알고있음. 여유를 좀 둠(@는 보통 전체 셋의 40%정도라고 하던데 여기는 그러면 너무 커져서 임의로 한건지 잘 모르겠... cifar10에서는 그러던데..)
cfg.QUEUE_CAPACITY = 100

# indicate if the model is in training mode #트레이닝이니
cfg.IS_TRAINING = True

#################### 위 까지가 기본 컨피그 ######################### 아래부터 모델별 컨피그 - kitti-squeezenet 기준으로 작성예정 ###############

#kitti, squeezenet 용으로 조금씩 변경함. 
cfg.IMAGE_WIDTH           = 1248
cfg.IMAGE_HEIGHT          = 384
cfg.BATCH_SIZE            = 20

cfg.WEIGHT_DECAY          = 0.0001
cfg.LEARNING_RATE         = 0.01
cfg.DECAY_STEPS           = 10000 #러닝레잇 리케이스텝
cfg.MAX_GRAD_NORM         = 1.0 #그래디언트 클리핑
cfg.MOMENTUM              = 0.9 #모멘텀
cfg.LR_DECAY_FACTOR       = 0.5 

#로스 코이피션트 뭔지 잘 모르곘 아마 로스식에서 람다 123 이거 해당하는 것으로 추정
cfg.LOSS_COEF_BBOX        = 5.0
cfg.LOSS_COEF_CONF_POS    = 75.0
cfg.LOSS_COEF_CONF_NEG    = 100.0
cfg.LOSS_COEF_CLASS       = 1.0

#각종 스레쉬홀드. 
cfg.PLOT_PROB_THRESH      = 0.4
cfg.NMS_THRESH            = 0.4
cfg.PROB_THRESH           = 0.005
cfg.TOP_N_DETECTION       = 64

#어규멘테이션을 하긴 하는구나 난하기싫은데... 
cfg.DATA_AUGMENTATION     = True
cfg.DRIFT_X               = 150
cfg.DRIFT_Y               = 100
cfg.EXCLUDE_HARD_EXAMPLES = False

cfg.ANCHOR_BOX            = set_anchors(cfg) #앵커 아래에
cfg.ANCHORS               = len(cfg.ANCHOR_BOX) #24 * 78 * 9
cfg.ANCHOR_PER_GRID       = 9 #그리드당 앵커는 9개 

#앵커 세팅. 
def set_anchors(cfg):
    H, W, B = 24, 78, 9
    anchor_shapes = np.reshape( #9개 앵커를 24 * 78개의 그리드에 
      [np.array(
          [[  36.,  37.], [ 366., 174.], [ 115.,  59.],
           [ 162.,  87.], [  38.,  90.], [ 258., 173.],
           [ 224., 108.], [  78., 170.], [  72.,  43.]])] * H * W,
      (H, W, B, 2)
    )
    center_x = np.reshape(
      np.transpose(
          np.reshape(
              np.array([np.arange(1, W+1)*float(cfg.IMAGE_WIDTH)/(W+1)]*H*B), 
              (B, H, W)
          ),
          (1, 2, 0)
      ),
      (H, W, B, 1)
    )
    center_y = np.reshape(
      np.transpose(
          np.reshape(
              np.array([np.arange(1, H+1)*float(cfg.IMAGE_HEIGHT)/(H+1)]*W*B),
              (B, W, H)
          ),
          (2, 1, 0)
      ),
      (H, W, B, 1)
    )
    anchors = np.reshape(
      np.concatenate((center_x, center_y, anchor_shapes), axis=3),
      (-1, 4)
    )

    return anchors

print('config done')

config done


# DATA REFINE, IMDB(KITTI)

In [17]:
#이거 두개만 imdb에서 써서 util에 있던거 그냥 복사해 옴
#imdb 만들때 GT를보고 만듦. GT데이터에 맞는 앵커, dx, dy 등을 세팅해줘야하기땜에.(코스트 계산하려면..)
# iou 계산 함수
def iou(box1, box2): #box1, box2 -> center x, y / width , height
    """Compute the Intersection-Over-Union of two given boxes.

    Args:
    box1: array of 4 elements [cx, cy, width, height].
    box2: same as above
    Returns:
    iou: a float number in range [0, 1]. iou of the two boxes.
    """
    # left-right? box1,2 중 작은 오른쪽 x좌표 - 큰 왼쪽 x좌표 -> 겹치는 x구간
    lr = min(box1[0]+0.5*box1[2], box2[0]+0.5*box2[2]) - \
        max(box1[0]-0.5*box1[2], box2[0]-0.5*box2[2])
    if lr > 0:
        # top-bottom? box1,2 중 작은 아래쪽 y좌표 - 큰 위쪽 y좌표 -> 겹치는 구간
        tb = min(box1[1]+0.5*box1[3], box2[1]+0.5*box2[3]) - \
        max(box1[1]-0.5*box1[3], box2[1]-0.5*box2[3])
        if tb > 0: # 둘다 0보다 크다면 겹치는 구간이 존재함. 
            intersection = tb*lr #겹치는 구간 면적 계산
            union = box1[2]*box1[3]+box2[2]*box2[3]-intersection # 전체 면적 계산

            return intersection/union #결과 리턴 
    return 0

#배치로 IOU 계산 > box2는 아마 GT가 들어올 것 같음. box1은 여러 anchor? 
def batch_iou(boxes, box):
    """Compute the Intersection-Over-Union of a batch of boxes with another
    box.

    Args:
    box1: 2D array of [cx, cy, width, height].
    box2: a single array of [cx, cy, width, height]
    Returns:
    ious: array of a float number in range [0, 1].
    """
    lr = np.maximum(#가장 lr이 큰 것 (0보다 큰 것 중)
        np.minimum(boxes[:,0]+0.5*boxes[:,2], box[0]+0.5*box[2]) - \
        np.maximum(boxes[:,0]-0.5*boxes[:,2], box[0]-0.5*box[2]),
        0
    )
    tb = np.maximum( #가장 tb가 큰 것 (0보다 큰 것 중)
        np.minimum(boxes[:,1]+0.5*boxes[:,3], box[1]+0.5*box[3]) - \
        np.maximum(boxes[:,1]-0.5*boxes[:,3], box[1]-0.5*box[3]),
        0
    )
    inter = lr*tb #동일
    union = boxes[:,2]*boxes[:,3] + box[2]*box[3] - inter 
    return inter/union

In [30]:
class imdb(object): #이미지 디비 - 카페에서 살짝쿵 빌려온 느낌? 아무튼 디텍션을 위한 준비. 
    """Image database."""
    """내맘대로 하면 되는데 보통 기본으로 이미지정보, 오브젝트 알오아이정보는 있어야 함. """

    def __init__(self, name, mc): #생성자 
        self._name = name #네임. 
        self._classes = [] #클래스
        self._image_set = [] #이미지셋
        self._image_idx = [] #이미지 인덱스
        self._data_root_path = [] #데이터 루트 패쓰
        self._rois = {} #알오아이(디텍션 좌표)
        self.mc = mc #엠 컨피규레이션

        # batch reader
        self._perm_idx = None #섞은인덱스
        self._cur_idx = 0 #커런트 인덱스 -> 둘다 배치용

    @property
    def name(self):
        return self._name

    @property
    def classes(self):
        return self._classes

    @property
    def num_classes(self):
        return len(self._classes)

    @property
    def image_idx(self):
        return self._image_idx

    @property
    def image_set(self):
        return self._image_set

    @property
    def data_root_path(self):
        return self._data_root_path

    @property
    def year(self):
        return self._year

    #셔플 이미지 인덱스. > 보통 셔플 돌리니까
    def _shuffle_image_idx(self): # 요 함수는 그냥 이미지 인덱스를 셔플해주고 시작 인덱스를 초기화(0)으로 해준다. 
        self._perm_idx = [self._image_idx[i] for i in #이미지 인덱스 전체를 랜덤으로 돌려서 넣어줌 갯수가 많아도 이렇게 똑같이 하나?? 
            np.random.permutation(np.arange(len(self._image_idx)))]
        self._cur_idx = 0 #시작은 0으로 맞춰두고 

     #리드 이미지 배치 - 이미지의 배치를 리드 한다. 
    def read_image_batch(self, shuffle=True): #이미지를 배치사이즈만큼 읽어온다. 셔플여부는 선택. 크기는 조절할수있음. 
        """Only Read a batch of images
        Args:
          shuffle: whether or not to shuffle the dataset
        Returns:
          images: length batch_size list of arrays [height, width, 3]
        """
        mc = cfg #config file. 각종 하이퍼파라미터 상수 등등이 다 요기 모델별로 다른건 다르게 같은건 같게 적혀있음. 
        if shuffle: #셔플을 하면
            if self._cur_idx + mc.BATCH_SIZE >= len(self._image_idx): #만약 배치사이즈+현재인덱스가 전체 갯수보다 많아지면 다시섞고,cur_idx = 0 
                self._shuffle_image_idx()
            batch_idx = self._perm_idx[self._cur_idx:self._cur_idx+mc.BATCH_SIZE] #배치인덱스는 섞어놓은거에서 배치만큼 가져온 인덱스들
            self._cur_idx += mc.BATCH_SIZE #커런트 인덱스 넘겨주고 
        else: #셔플을 안하면
            if self._cur_idx + mc.BATCH_SIZE >= len(self._image_idx): #만약 마지막 배치면 
                batch_idx = self._image_idx[self._cur_idx:] \
                    + self._image_idx[:self._cur_idx + mc.BATCH_SIZE-len(self._image_idx)] #배치 인덱스 끝 나머지 + 다시 처음으로 돌아와서 나머지 한마디로 사이클링
                self._cur_idx += mc.BATCH_SIZE - len(self._image_idx) #커런트 인덱스 대충 해줌 
            else: #평소는 그냥 배치사이즈만큼 차례대로 해줌 
                batch_idx = self._image_idx[self._cur_idx:self._cur_idx+mc.BATCH_SIZE]
                self._cur_idx += mc.BATCH_SIZE

        images, scales = [], [] #이미지와 스케일 
        for i in batch_idx: #배치인덱스 
            im = cv2.imread(self._image_path_at(i)) # 시브이로 아이엠 리드해옴. 이미지패스에서 매치인덱스 받아서. 
            im = im.astype(np.float32, copy=False) #타입 플롯32로 바꿔줌 
            im -= mc.BGR_MEANS #BGR MEAN을 뺴줌.  cfg.BGR_MEANS = np.array([[[103.939, 116.779, 123.68]]]) ? 이미지넷데이터에서 뽑은건가.. 
            orig_h, orig_w, _ = [float(v) for v in im.shape] #불러온 이미지 h w c 중에 h w 받아옴 
            im = cv2.resize(im, (mc.IMAGE_WIDTH, mc.IMAGE_HEIGHT)) #이미지 리사이즈 해줌 
            #mc.IMAGE_WIDTH           = 1248 # half width 621
            #mc.IMAGE_HEIGHT          = 384 # half height 187
            x_scale = mc.IMAGE_WIDTH/orig_w #스케일팩터 구함
            y_scale = mc.IMAGE_HEIGHT/orig_h
            images.append(im) #이미지스에 넣어줌.
            scales.append((x_scale, y_scale)) #스케일스에 넣어줌. 

        return images, scales #이미지와 스케일을 뱉어냄 

    def read_batch(self, shuffle=True): #배치. 바운딩박스 어노테이션이랑 이미지 배치로읽어옴 
        """Read a batch of image and bounding box annotations.
        Args:
          shuffle: whether or not to shuffle the dataset
        Returns:
          image_per_batch: images. Shape: batch_size x width x height x [b, g, r]
          label_per_batch: labels. Shape: batch_size x object_num
          delta_per_batch: bounding box deltas. Shape: batch_size x object_num x 
              [dx ,dy, dw, dh]
          aidx_per_batch: index of anchors that are responsible for prediction.
              Shape: batch_size x object_num
          bbox_per_batch: scaled bounding boxes. Shape: batch_size x object_num x 
              [cx, cy, w, h]
        """
        mc = self.mc #config

        if shuffle:#셔플이면 위에 리드이미지랑 똑가틈 
            if self._cur_idx + mc.BATCH_SIZE >= len(self._image_idx):
                self._shuffle_image_idx()
            batch_idx = self._perm_idx[self._cur_idx:self._cur_idx+mc.BATCH_SIZE]
            self._cur_idx += mc.BATCH_SIZE
        else:#아니어도 똑가틈 
            if self._cur_idx + mc.BATCH_SIZE >= len(self._image_idx):
                batch_idx = self._image_idx[self._cur_idx:] \
                    + self._image_idx[:self._cur_idx + mc.BATCH_SIZE-len(self._image_idx)]
                self._cur_idx += mc.BATCH_SIZE - len(self._image_idx)
            else:
                batch_idx = self._image_idx[self._cur_idx:self._cur_idx+mc.BATCH_SIZE]
                self._cur_idx += mc.BATCH_SIZE
                
        image_per_batch = [] #이미지
        label_per_batch = [] #라벨
        bbox_per_batch  = [] #바운딩박스
        delta_per_batch = [] #델타
        aidx_per_batch  = [] # a index 
        if mc.DEBUG_MODE:#디버그 모드 용 
            avg_ious = 0.  #아이오유 평균 
            num_objects = 0. #오브젝트수
            max_iou = 0.0 #최대 아이오유
            min_iou = 1.0 #최소 아이오유
            num_zero_iou_obj = 0 #아이오유 0인 오브젝트? 

        for idx in batch_idx: #배치
            # load the image
            im = cv2.imread(self._image_path_at(idx)).astype(np.float32, copy=False) #이미지 열어서 타입 변환 
            im -= mc.BGR_MEANS #비지알민 뺴줌
            orig_h, orig_w, _ = [float(v) for v in im.shape] #h w 받아옴

            # load annotations 어노테이션도 로드해줌 
            label_per_batch.append([b[4] for b in self._rois[idx][:]]) #라벨퍼 배치에 
            gt_bbox = np.array([[b[0], b[1], b[2], b[3]] for b in self._rois[idx][:]]) #센타 , w h 임 

            if mc.DATA_AUGMENTATION: #데이터 늘리기 할꺼면 
                assert mc.DRIFT_X >= 0 and mc.DRIFT_Y > 0, \
                    'mc.DRIFT_X and mc.DRIFT_Y must be >= 0' #드리프트 x y 둘다 당근 0 이상이어야 함 
            # The range to randomly shift the image widht & height라고함
            #cfg.DRIFT_X = 0
            #cfg.DRIFT_Y = 0
            #이지만 kitti는 150정도 쓰는걸 보니 좀 오규멘테이션 하긴 하는듯. 

                if mc.DRIFT_X > 0 or mc.DRIFT_Y > 0: #만약에 0보다 크면 
                    # Ensures that gt boundibg box is not cutted out of the image
                    max_drift_x = min(gt_bbox[:, 0] - gt_bbox[:, 2]/2.0+1) #순서가... cx cy w h > left top 구함
                    max_drift_y = min(gt_bbox[:, 1] - gt_bbox[:, 3]/2.0+1) #최대로 옮길수 있는(좌측, 위쪽으로) 크기 구하는거임.
                    assert max_drift_x >= 0 and max_drift_y >= 0, 'bbox out of image' #0보다 크면... 무슨 의미인지 잘 모르겠다. 
                    #만약 작다고 치면 바운딩박스를 못움직인다는거 같은데.. 그럼 오규멘테이션 불가. 

                    dy = np.random.randint(-mc.DRIFT_Y, min(mc.DRIFT_Y+1, max_drift_y)) #아무튼 움직일수 있다면...  랜덤을 -~+ 사이에 만듬 
                    dx = np.random.randint(-mc.DRIFT_X, min(mc.DRIFT_X+1, max_drift_x))
                    #움직임 가능하면 움직이는데... +방향은 바운더리 설정을 안해줘서 이건 코드 오류같음. right bottom도 구해야하는데.

                    # shift bbox
                    gt_bbox[:, 0] = gt_bbox[:, 0] - dx #실제 지티박스 이동함. 쉬프팅 
                    gt_bbox[:, 1] = gt_bbox[:, 1] - dy

                    # distort image
                    orig_h -= dy #이미지 옮겼으니 오리지널 이미지에서 gt도 바까줌. 
                    orig_w -= dx 
                    orig_x, dist_x = max(dx, 0), max(-dx, 0) #이건 여기서 만든 변수인듯 위에 없네  0이랑 dx -dx 해서 아무튼 -dx~dx
                    orig_y, dist_y = max(dy, 0), max(-dy, 0) #delta가 +일 경우 오리지날은 + 디스토션은 0 / -는 반대 

                    distorted_im = np.zeros(
                    (int(orig_h), int(orig_w), 3)).astype(np.float32) #디스토션된 빈 이미지를 만들어둠.
                    distorted_im[dist_y:, dist_x:, :] = im[orig_y:, orig_x:, :] #바뀐 이미지 좌표에 넣는데... 
                    im = distorted_im#아무튼 이미지에 디스토션된거 넣음 

                # Flip image with 50% probability
                if np.random.randint(2) > 0.5: #50%프로 확률로 플립을 이렇게 구현하다니 대단하다 0이냐 1이냐 그거시문제구먼
                    im = im[:, ::-1, :] #좌우플립
                    gt_bbox[:, 0] = orig_w - 1 - gt_bbox[:, 0] #좌우플립하니까 width -1- minx??? 이거만 바꾸면 되나.. 그럼 좌표계가 minmax가아닌가

            # scale image
            im = cv2.resize(im, (mc.IMAGE_WIDTH, mc.IMAGE_HEIGHT)) #이미지 리사이즈로 인풋에 맞게 조정하는듯. 
            image_per_batch.append(im) 

            # scale annotation
            x_scale = mc.IMAGE_WIDTH/orig_w #스케일 정보 > 정해진 이미지 / 오리지날 쉐입. 아마 kitti에 맞춰놔서 1일거임 
            y_scale = mc.IMAGE_HEIGHT/orig_h
            gt_bbox[:, 0::2] = gt_bbox[:, 0::2]*x_scale #두개씩 띄워서.. 이거 보면 minx maxx 맞는거같기도 한데.. x, width로도 상관없긴 한듯 
            gt_bbox[:, 1::2] = gt_bbox[:, 1::2]*y_scale 
            bbox_per_batch.append(gt_bbox)

            aidx_per_image, delta_per_image = [], [] #앵커인덱스 / 델타는 옮겨진거 dx dy 이거 
            aidx_set = set() # aidx를 셋으로 쓸 일이 있나봄. 

            for i in range(len(gt_bbox)): #그라운드트루쓰 바운딩박스 갯수를 전체 돌면서 
                overlaps = batch_iou(mc.ANCHOR_BOX, gt_bbox[i]) #iou를 해당 gt박스에 대해 모든 앵커박스랑 계산해본다 

                aidx = len(mc.ANCHOR_BOX) #a idx는 앵커 인덱스
                for ov_idx in np.argsort(overlaps)[::-1]: #arg sort로 아규먼트 소팅 해준 다음에 반전 >> 가장 iou높은 인덱스부터 뽑아옴 
                    if overlaps[ov_idx] <= 0: #오버랩된게 0보다 작으면??? 작은게 가능한가??? 
                        if mc.DEBUG_MODE: #디버깅할때쓰고
                            min_iou = min(overlaps[ov_idx], min_iou)
                            num_objects += 1
                            num_zero_iou_obj += 1
                        break
                    if ov_idx not in aidx_set: #a idx set에 오버랩 인덱스가 없다면 
                        aidx_set.add(ov_idx) #추가해준다. 
                        aidx = ov_idx # 요 인덱스를 앵커인덱스로 만든다..
                        if mc.DEBUG_MODE: #디버깅시
                            max_iou = max(overlaps[ov_idx], max_iou)
                            min_iou = min(overlaps[ov_idx], min_iou)
                            avg_ious += overlaps[ov_idx]
                            num_objects += 1
                        break

                if aidx == len(mc.ANCHOR_BOX):  #만약 이렇다면 초기값이 유지된다는거니 오버랩된것이 하나도 없다는 것? 인듯. 
                    # even the largeset available overlap is 0, thus, choose one with the
                    # smallest square distance
                    dist = np.sum(np.square(gt_bbox[i] - mc.ANCHOR_BOX), axis=1) # 스퀘어 에러 썸 > 앵커랑 그라운드트루쓰랑 
                    for dist_idx in np.argsort(dist): #소팅한다음에... 가장 작은놈의 아규먼트부터 
                        if dist_idx not in aidx_set: #셋에 없으면 셋에 넣어줌. 
                            aidx_set.add(dist_idx)
                            aidx = dist_idx
                            break

                #어쨋든 그라운드트루쓰 vs 앵커박스를 해서 > iou높은순서대로 하는데 만약 다 오버랩 안되면 그나마 스퀘어에러가 작은놈부터 넣어준다는건가

                box_cx, box_cy, box_w, box_h = gt_bbox[i] #센터 기준 좌표 
                delta = [0]*4 #델타 만들어줌 
                delta[0] = (box_cx - mc.ANCHOR_BOX[aidx][0])/mc.ANCHOR_BOX[aidx][2] #이 과정은 박스 트랜스폼해주는 과정? 앵커랑 박스랑 미세조절? 
                delta[1] = (box_cy - mc.ANCHOR_BOX[aidx][1])/mc.ANCHOR_BOX[aidx][3]
                delta[2] = np.log(box_w/mc.ANCHOR_BOX[aidx][2])
                delta[3] = np.log(box_h/mc.ANCHOR_BOX[aidx][3])
                # 위의 포문이 왜 필요한건지... 이런게 궁금하네.. 

                aidx_per_image.append(aidx) #이미지당 앵커인덱스??? 
                delta_per_image.append(delta)#이미지당 델타? 

            delta_per_batch.append(delta_per_image)
            aidx_per_batch.append(aidx_per_image) #배치에 넣어줌. 

        if mc.DEBUG_MODE: #디버그 모드에서는 뭐 이런거 출력도 해줌. 
            print ('max iou: {}'.format(max_iou))
            print ('min iou: {}'.format(min_iou))
            print ('avg iou: {}'.format(avg_ious/num_objects))
            print ('number of objects: {}'.format(num_objects))
            print ('number of objects with 0 iou: {}'.format(num_zero_iou_obj))

        return image_per_batch, label_per_batch, delta_per_batch, \
            aidx_per_batch, bbox_per_batch #이미지, 라벨, 델타, 앵커인덱스, 바운딩박스를 배치로 리턴해줌 

In [ ]:
#UTIL 
def bbox_transform(bbox):
    """convert a bbox of form [cx, cy, w, h] to [xmin, ymin, xmax, ymax]. Works
    for numpy array or list of tensors.
    """
    with tf.variable_scope('bbox_transform') as scope:
        cx, cy, w, h = bbox
        out_box = [[]]*4
        out_box[0] = cx-w/2
        out_box[1] = cy-h/2
        out_box[2] = cx+w/2
        out_box[3] = cy+h/2

    return out_box

def bbox_transform_inv(bbox):
    """convert a bbox of form [xmin, ymin, xmax, ymax] to [cx, cy, w, h]. Works
    for numpy array or list of tensors.
    """
    with tf.variable_scope('bbox_transform_inv') as scope:
        xmin, ymin, xmax, ymax = bbox
        out_box = [[]]*4

        width       = xmax - xmin + 1.0
        height      = ymax - ymin + 1.0
        out_box[0]  = xmin + 0.5*width 
        out_box[1]  = ymin + 0.5*height
        out_box[2]  = width
        out_box[3]  = height

    return out_box

# KITTI DATASET용
"""Image data base class for kitti"""

class kitti(imdb): #키티 데이터셋 아이엠디비 상속해서 맹듬 
    def __init__(self, image_set, data_path, mc):
        imdb.__init__(self, 'kitti_'+image_set, mc) #아이엠디비 이닛 이름은 저렇게 줌. 
        self._image_set = image_set
        self._data_root_path = data_path
        self._image_path = os.path.join(self._data_root_path, 'training', 'image_2') #이미지
        self._label_path = os.path.join(self._data_root_path, 'training', 'label_2') #라벨
        self._classes = self.mc.CLASS_NAMES
        self._class_to_idx = dict(zip(self.classes, xrange(self.num_classes)))

        # a list of string indices of images in the directory
        self._image_idx = self._load_image_set_idx() 
        # a dict of image_idx -> [[cx, cy, w, h, cls_idx]]. x,y,w,h are not divided by #오 이게 진짜 필요한거였는데. 이거인듯. 
        # the image width and height
        self._rois = self._load_kitti_annotation() #알오아이를 위와같은 포멧으로 만드러쥼. 고 함수는 아래에... 

        ## batch reader ##
        self._perm_idx = None
        self._cur_idx = 0
        # TODO(bichen): add a random seed as parameter
        self._shuffle_image_idx()

        self._eval_tool = './src/dataset/kitti-eval/cpp/evaluate_object' #이건 뭐 언제 쓰이나 잘 모르겠.. 

    def _load_image_set_idx(self): #이미지를 로드하고.. 이미지인덱스를 세팅. 
        image_set_file = os.path.join(self._data_root_path, 'ImageSets', self._image_set+'.txt')
        assert os.path.exists(image_set_file), \
            'File does not exist: {}'.format(image_set_file)

        with open(image_set_file) as f:
            image_idx = [x.strip() for x in f.readlines()]
        return image_idx

    def _image_path_at(self, idx):#패스랑 인덱스 받아와서 있나없나... 만들어줌 
        image_path = os.path.join(self._image_path, idx+'.png')  
        assert os.path.exists(image_path), \
            'Image does not exist: {}'.format(image_path)
        return image_path

    def _load_kitti_annotation(self): #키티 어노테이션을 만들어준다. 
        def _get_obj_level(obj): #오브젝트 레벨 받아오는거. -> 이건 그냥 트렁케이션, 오클루션가지고 어려움 난이도 세팅해주는듯. 
            height = float(obj[7]) - float(obj[5]) + 1 #obj에 있는 정보 가지고 이런거 저런거 해주는거임 그냥.. 
            truncation = float(obj[1])
            occlusion = float(obj[2])
            if height >= 40 and truncation <= 0.15 and occlusion <= 0:
                return 1
            elif height >= 25 and truncation <= 0.3 and occlusion <= 1:
                return 2
            elif height >= 25 and truncation <= 0.5 and occlusion <= 2:
                return 3
            else:
                return 4

        idx2annotation = {} #인덱스 투 어노테이션 
        for index in self._image_idx:
            filename = os.path.join(self._label_path, index+'.txt') #라벨 뽑아내는 과정인듯.. 어노테이션 
            with open(filename, 'r') as f: #라인 다 읽어옴
                lines = f.readlines()
            f.close()
            bboxes = []
            
            #0000002.txt의 예.. 
            #Misc 0.00 0 -1.82 804.79 167.34 995.43 327.94 1.63 1.48 2.37 3.23 1.59 8.55 -1.47
            #Car 0.00 0 -1.67 657.39 190.13 700.07 223.39 1.41 1.58 4.36 3.18 2.27 34.38 -1.58
            #0 - cls / 1 - truncation / 2 - occlusion / 5 - miny / 7 - maxy / 4 - minx / 6 - maxx
            
            for line in lines:
                obj = line.strip().split(' ') # 양옆 빈칸다지우고 ' ' 로 나눔 그럼 이게 바로 각각 오브젝트 정보가 됨. 
                try:
                    cls = self._class_to_idx[obj[0].lower().strip()] #클래스는 다 lowercase로 해주고 스트립 함. 그리고 젤 0번 
                except:
                    continue

                if self.mc.EXCLUDE_HARD_EXAMPLES and _get_obj_level(obj) > 3: #레벨이 3보다 높고 하드한 예제를 포기하기로 했으면 버림 아니면
                    continue
                xmin = float(obj[4])
                ymin = float(obj[5])
                xmax = float(obj[6])
                ymax = float(obj[7]) #해당 roi 좌표 찍어넣음. 
                assert xmin >= 0.0 and xmin <= xmax, \
                    'Invalid bounding box x-coord xmin {} or xmax {} at {}.txt' \
                        .format(xmin, xmax, index)
                assert ymin >= 0.0 and ymin <= ymax, \
                    'Invalid bounding box y-coord ymin {} or ymax {} at {}.txt' \
                        .format(ymin, ymax, index)
                x, y, w, h = bbox_transform_inv([xmin, ymin, xmax, ymax]) #이건 그냥 센터 x,y, w h로 바꿔줌. 
                bboxes.append([x, y, w, h, cls]) #바운딩박스 어펜드 해줌 

            idx2annotation[index] = bboxes  #해당 인덱스에 박스 넣어줌. > 좌표 + 클래스 
        return idx2annotation # 리턴 

    def evaluate_detections(self, eval_dir, global_step, all_boxes):#디텍션 결과를 평가함. 
        """Evaluate detection results.
        Args:
          eval_dir: directory to write evaluation logs
          global_step: step of the checkpoint
          all_boxes: all_boxes[cls][image] = N x 5 arrays of 
            [xmin, ymin, xmax, ymax, score]
        Returns:
          aps: array of average precisions.
          names: class names corresponding to each ap
        """
        det_file_dir = os.path.join(
            eval_dir, 'detection_files_{:s}'.format(global_step), 'data') #평가용 파일  디렉토리 
        if not os.path.isdir(det_file_dir):
            os.makedirs(det_file_dir)

        for im_idx, index in enumerate(self._image_idx): #이미지 인덱스 따라 
            filename = os.path.join(det_file_dir, index+'.txt') #또 라벨 
            with open(filename, 'wt') as f:
                for cls_idx, cls in enumerate(self._classes): #라벨 하나씩... 
                    dets = all_boxes[cls_idx][im_idx] #모든박스 -> 받아온데서 클래스인덱스 / 이미지 인덱스 받아온거 ??? 뭐 어떻게...
                    for k in xrange(len(dets)): #아무튼 ... 클래스 / 이미지를 키로 가져온거는 위에 처럼 코티네잇 + 컨피던스스코어로 보임
                        f.write(
                            '{:s} -1 -1 0.0 {:.2f} {:.2f} {:.2f} {:.2f} 0.0 0.0 0.0 0.0 0.0 '
                            '0.0 0.0 {:.3f}\n'.format(
                                cls.lower(), dets[k][0], dets[k][1], dets[k][2], dets[k][3],
                                dets[k][4])
                        ) #이건 라벨을 쓰는건가... 근데 GT는 아닐껀데... 그냥 로그를 쓰는게 아닌가 싶다. 필요없는건 다 0000 

        cmd = self._eval_tool + ' ' \ 
            + os.path.join(self._data_root_path, 'training') + ' ' \
            + os.path.join(self._data_root_path, 'ImageSets',
                         self._image_set+'.txt') + ' ' \
            + os.path.dirname(det_file_dir) + ' ' + str(len(self._image_idx)) #커맨드를 입력하는 것 같은데... 

        print('Running: {}'.format(cmd))
        status = subprocess.call(cmd, shell=True) #실제 커맨드를 써서 뭘 eval하려는건가... 여기는 실제 실행부를 보던지 해야할듯.. 

        aps = []
        names = []
        for cls in self._classes: 
            det_file_name = os.path.join(
                os.path.dirname(det_file_dir), 'stats_{:s}_ap.txt'.format(cls)) #뭐지.. 모르겠으나 누가 만들어놨다거나 뭔가 있다면 읽어오는듯 
            if os.path.exists(det_file_name):
                with open(det_file_name, 'r') as f:
                    lines = f.readlines()
                assert len(lines) == 3, \
                    'Line number of {} should be 3'.format(det_file_name)

                aps.append(float(lines[0].split('=')[1].strip()))
                aps.append(float(lines[1].split('=')[1].strip()))
                aps.append(float(lines[2].split('=')[1].strip()))
            else: #그런거 없다면 aps는 0 0 0 
                aps.extend([0.0, 0.0, 0.0])

            names.append(cls+'_easy')
            names.append(cls+'_medium')
            names.append(cls+'_hard') #이지 미듐 하드를 추가해줌 네임에.. 왜? 

        return aps, names

    def do_detection_analysis_in_eval(self, eval_dir, global_step): #평가에서 디텍션 분석을 함?? 
        det_file_dir = os.path.join(
            eval_dir, 'detection_files_{:s}'.format(global_step), 'data') #데이터
        det_error_dir = os.path.join(
            eval_dir, 'detection_files_{:s}'.format(global_step), 'error_analysis') #에러분석
        if not os.path.exists(det_error_dir):
            os.makedirs(det_error_dir)
        det_error_file = os.path.join(det_error_dir, 'det_error_file.txt') #에러 파일인데... 도대체 어따쓸라거

        stats = self.analyze_detections(det_file_dir, det_error_file) #아래 함수임. 오류난거 분석
        ims = self.visualize_detections( 
            image_dir=self._image_path,
            image_format='.png',
            det_error_file=det_error_file,
            output_image_dir=det_error_dir,
            num_det_per_type=10
        )#이건 디텍션 비주얼라이즈 해주는듯

        return stats, ims


    def analyze_detections(self, detection_file_dir, det_error_file): #디텍션을 분석해주는듯 
        def _save_detection(f, idx, error_type, det, score): #세이브 해줌. 아래같은 포맷으로.. 
            f.write(
              '{:s} {:s} {:.1f} {:.1f} {:.1f} {:.1f} {:s} {:.3f}\n'.format(
                  idx, error_type,
                  det[0]-det[2]/2., det[1]-det[3]/2.,
                  det[0]+det[2]/2., det[1]+det[3]/2.,
                  self._classes[int(det[4])], 
                  score
                )
            )

        # load detections #디텍션을 로드하준다는게 뭐 쥐
        self._det_rois = {}
        for idx in self._image_idx: #아뮤튼 인덱스 돌면서 
            det_file_name = os.path.join(detection_file_dir, idx+'.txt') #텍스트니까 라벨 가져오는거겟지 뭐 
            with open(det_file_name) as f:
                lines = f.readlines()
            f.close()
            bboxes = []
            for line in lines:
                obj = line.strip().split(' ')
                cls = self._class_to_idx[obj[0].lower().strip()] #클래스 0번 
                xmin = float(obj[4])# xmin 4
                ymin = float(obj[5])# ymin 5
                xmax = float(obj[6])# xmax 6
                ymax = float(obj[7])# ymax 7
                score = float(obj[-1]) #마지막껀 스코어 

                x, y, w, h = bbox_transform_inv([xmin, ymin, xmax, ymax]) #바꿔줌. 
                bboxes.append([x, y, w, h, cls, score])
            bboxes.sort(key=lambda x: x[-1], reverse=True) #리버스 소팅??? 뭘 기준으로? x기준으로? 
            self._det_rois[idx] = bboxes #디텍션 알오아이 

        # do error analysis #에러 분석 에러 갯수 등등이 있음. 
        num_objs = 0.
        num_dets = 0.
        num_correct = 0.
        num_loc_error = 0.
        num_cls_error = 0.
        num_bg_error = 0.
        num_repeated_error = 0.
        num_detected_obj = 0.

        with open(det_error_file, 'w') as f: #에러 파일을 오픈해서 
            for idx in self._image_idx: #인덱스 돌면서 
                gt_bboxes = np.array(self._rois[idx]) # 이미지에 알오아이들임 
                num_objs += len(gt_bboxes) #오브젝트는 지티박스 수임 각 이미지별 
                detected = [False]*len(gt_bboxes)# 하나도 디텍트되지않았나봄아직 

                det_bboxes = self._det_rois[idx] #디텍션 바운딩박스 -> 디텍션 알오아이들 
                if len(gt_bboxes) < 1:  #지티바운딩박스가 읎으면? 다음
                    continue

                for i, det in enumerate(det_bboxes): #다택션 한 바운딩박스를 가지고 하는거같음. 
                    if i < len(gt_bboxes): #지티 바운딩박스만큼 + 해줌. 그럼 FP가 많으면 막 +++++ 되지 않나... 무조건 100%막 
                        num_dets += 1
                    ious = batch_iou(gt_bboxes[:, :4], det[:4]) #배치로 iou계산함 
                    max_iou = np.max(ious)  #맥스
                    gt_idx = np.argmax(ious) #맥스
                    if max_iou > 0.1: #맥스 아이오유 0.1 이상이면 
                        if gt_bboxes[gt_idx, 4] == det[4]: #?? 음... 잘... 모르겟...  아래를 보다보니 cls인듯...
                            if max_iou >= 0.5: #0.5도 넘으면 
                                if i < len(gt_bboxes):
                                    if not detected[gt_idx]: #이 쥐티가 디텍된게 아니면 
                                        num_correct += 1
                                        detected[gt_idx] = True #디텍된거 처리헤줌
                                    else:
                                        num_repeated_error += 1 #한 지티에 여러개 잡은거 에러 ++ 
                            else:
                                if i < len(gt_bboxes): 
                                    num_loc_error += 1#로컬라이제이션 에러 난듯 
                                    _save_detection(f, idx, 'loc', det, det[5])#에러 세이브 해줌 
                        else:
                            if i < len(gt_bboxes): 
                                num_cls_error += 1#이번엔 클래스 에러임 
                                _save_detection(f, idx, 'cls', det, det[5])
                    else: #0.1도 안되면... 
                        if i < len(gt_bboxes):
                            num_bg_error += 1 #bg에러? 백그라운드?? ㅇ..
                            _save_detection(f, idx, 'bg', det, det[5])

                for i, gt in enumerate(gt_bboxes):
                    if not detected[i]: #디텍 안된 지티는 다 미스에러 
                        _save_detection(f, idx, 'missed', gt, -1.0)
                num_detected_obj += sum(detected)#디텍된 오브젝트는 요기에 저장 
            f.close()

            print ('Detection Analysis:')
            print ('    Number of detections: {}'.format(num_dets))
            print ('    Number of objects: {}'.format(num_objs))
            print ('    Percentage of correct detections: {}'.format(num_correct/num_dets))
            print ('    Percentage of localization error: {}'.format(num_loc_error/num_dets))
            print ('    Percentage of classification error: {}'.format(num_cls_error/num_dets))
            print ('    Percentage of background error: {}'.format(num_bg_error/num_dets))
            print ('    Percentage of repeated detections: {}'.format(num_repeated_error/num_dets))
            print ('    Recall: {}'.format(num_detected_obj/num_objs))

            out = {}
            out['num of detections'] = num_dets
            out['num of objects'] = num_objs
            out['% correct detections'] = num_correct/num_dets
            out['% localization error'] = num_loc_error/num_dets
            out['% classification error'] = num_cls_error/num_dets
            out['% background error'] = num_bg_error/num_dets
            out['% repeated error'] = num_repeated_error/num_dets
            out['% recall'] = num_detected_obj/num_objs

            return out


# DB, CONFIG 준비 끝
# 이제 실제 데이터 불러와서 시작해보자

In [ ]:
def _conv_layer(self, layer_name, inputs, filters, size, stride, padding='SAME', freeze=False, xavier=False, relu=True, stddev=0.001):
    """Convolutional layer operation constructor.

    Args:
      layer_name: layer name.
      inputs: input tensor
      filters: number of output filters.
      size: kernel size.
      stride: stride
      padding: 'SAME' or 'VALID'. See tensorflow doc for detailed description.
      freeze: if true, then do not train the parameters in this layer.
      xavier: whether to use xavier weight initializer or not.
      relu: whether to use relu or not.
      stddev: standard deviation used for random weight initializer.
    Returns:
      A convolutional layer operation.
    """

    mc = self.mc
    use_pretrained_param = False
    if mc.LOAD_PRETRAINED_MODEL: #만약 프리트레인된 모델을 쓸꺼라면... 난 안쓸거같긴한데쓸수도있고 
        cw = self.caffemodel_weight
        if layer_name in cw:
            kernel_val = np.transpose(cw[layer_name][0], [2,3,1,0]) # 인 아웃 세로가로를 세로가로인아웃으로 바까줌. 카페와 텐서플로의 차이 
            bias_val = cw[layer_name][1]
        # check the shape
        if (kernel_val.shape == 
              (size, size, inputs.get_shape().as_list()[-1], filters)) \
           and (bias_val.shape == (filters, )):
                use_pretrained_param = True
        else:
            print ('Shape of the pretrained parameter of {} does not match, '
              'use randomly initialized parameter'.format(layer_name))
    else: #프리트레인모델을 사용하지 않은 경우... 
        print ('Cannot find {} in the pretrained model. Use randomly initialized '
               'parameters'.format(layer_name))

    if mc.DEBUG_MODE: #디버그 모드라면. 
        print('Input tensor shape to {}: {}'.format(layer_name, inputs.get_shape()))

    with tf.variable_scope(layer_name) as scope:
        channels = inputs.get_shape()[3]
        # re-order the caffe kernel with shape [out, in, h, w] -> tf kernel with
        # shape [h, w, in, out]
        if use_pretrained_param: #프레인된 파라미터를 쓴다 하면 
            if mc.DEBUG_MODE:
                print ('Using pretrained model for {}'.format(layer_name))
            kernel_init = tf.constant(kernel_val , dtype=tf.float32)
            bias_init = tf.constant(bias_val, dtype=tf.float32)
        elif xavier:
            kernel_init = tf.contrib.layers.xavier_initializer_conv2d()
            bias_init = tf.constant_initializer(0.0)
        else:
            kernel_init = tf.truncated_normal_initializer(
                stddev=stddev, dtype=tf.float32)
            bias_init = tf.constant_initializer(0.0)

        kernel = _variable_with_weight_decay(
              'kernels', shape=[size, size, int(channels), filters],
              wd=mc.WEIGHT_DECAY, initializer=kernel_init, trainable=(not freeze))

        biases = _variable_on_device('biases', [filters], bias_init, 
                                    trainable=(not freeze))
        self.model_params += [kernel, biases]

        conv = tf.nn.conv2d(
              inputs, kernel, [1, stride, stride, 1], padding=padding,
              name='convolution')
        conv_bias = tf.nn.bias_add(conv, biases, name='bias_add')
  
        if relu:
            out = tf.nn.relu(conv_bias, 'relu')
        else:
            out = conv_bias

        self.model_size_counter.append(
              (layer_name, (1+size*size*int(channels))*filters)
        )
        out_shape = out.get_shape().as_list()
        num_flops = (1+2*int(channels)*size*size)*filters*out_shape[1]*out_shape[2]
        if relu:
            num_flops += 2*filters*out_shape[1]*out_shape[2]
        self.flop_counter.append((layer_name, num_flops))
        self.activation_counter.append((layer_name, out_shape[1]*out_shape[2]*out_shape[3]))
        return out
    
def _pooling_layer(self, layer_name, inputs, size, stride, padding='SAME'):
    """Pooling layer operation constructor.

    Args:
      layer_name: layer name.
      inputs: input tensor
      size: kernel size.
      stride: stride
      padding: 'SAME' or 'VALID'. See tensorflow doc for detailed description.
    Returns:
      A pooling layer operation.
    """

    with tf.variable_scope(layer_name) as scope:
        out =  tf.nn.max_pool(inputs,
                              ksize=[1, size, size, 1], 
                              strides=[1, stride, stride, 1],
                              padding=padding)
        activation_size = np.prod(out.get_shape().as_list()[1:])
        self.activation_counter.append((layer_name, activation_size))
        return out

  

In [ ]:
def _fire_layer(self, layer_name, inputs, s1x1, e1x1, e3x3, stddev=0.01,
      freeze=False):
        """Fire layer constructor.

        Args:
          layer_name: layer name
          inputs: input tensor
          s1x1: number of 1x1 filters in squeeze layer.
          e1x1: number of 1x1 filters in expand layer.
          e3x3: number of 3x3 filters in expand layer.
          freeze: if true, do not train parameters in this layer.
        Returns:
          fire layer operation.
        """

        sq1x1 = self._conv_layer(
            layer_name+'/squeeze1x1', inputs, filters=s1x1, size=1, stride=1,
            padding='SAME', stddev=stddev, freeze=freeze)
        ex1x1 = self._conv_layer(
            layer_name+'/expand1x1', sq1x1, filters=e1x1, size=1, stride=1,
            padding='SAME', stddev=stddev, freeze=freeze)
        ex3x3 = self._conv_layer(
            layer_name+'/expand3x3', sq1x1, filters=e3x3, size=3, stride=1,
            padding='SAME', stddev=stddev, freeze=freeze)

        return tf.concat([ex1x1, ex3x3], 3, name=layer_name+'/concat')

In [ ]:
"""SqueezeDet model.""" #변경 고민중... INIT은 따로 놓고. 모델만 SQUUEZEDET으로.. 

def SqueezeDet(mc):
    
    ###############################INIT #####################################
    self.mc = mc #model config 넣어줌
    # a scalar tensor in range (0, 1]. Usually set to 0.5 in training phase and
    # 1.0 in evaluation phase
    self.keep_prob = 0.5 if mc.IS_TRAINING else 1.0 #머지. 드랍아웃 아닌가 
    
    # image batch input #이미지 배치 인풋을 플레이스홀더로 만들어줌 
    self.ph_image_input = tf.placeholder(
        tf.float32, [mc.BATCH_SIZE, mc.IMAGE_HEIGHT, mc.IMAGE_WIDTH, 3],
        name='image_input'
    )
    # A tensor where an element is 1 if the corresponding box is "responsible" #플레이스홀더들. 인풋 마스크, 델타인풋, 박스인풋, 라벨
    # for detection an object and 0 otherwise.
    self.ph_input_mask = tf.placeholder(
        tf.float32, [mc.BATCH_SIZE, mc.ANCHORS, 1], name='box_mask') #배치사이즈 * 앵커 * 1 > 디텍션시 1 아니면 0 
    # Tensor used to represent bounding box deltas.
    self.ph_box_delta_input = tf.placeholder(
        tf.float32, [mc.BATCH_SIZE, mc.ANCHORS, 4], name='box_delta_input') #배치 * 앵커 * 4 > 바운딩박스 코디네잇 델타 
    # Tensor used to represent bounding box coordinates.
    self.ph_box_input = tf.placeholder(
        tf.float32, [mc.BATCH_SIZE, mc.ANCHORS, 4], name='box_input') #배치 * 앵커 * 4 > 바운딩박스 코디네잇 
    # Tensor used to represent labels 
    self.ph_labels = tf.placeholder(
        tf.float32, [mc.BATCH_SIZE, mc.ANCHORS, mc.CLASSES], name='labels')  # 배치 * 앵커 * 클래스들 > 클래스별 prob 

    # IOU between predicted anchors with ground-truth boxes # IOU는 배리어블로. 배치사이즈 * 앵커 
    self.ious = tf.Variable(
      initial_value=np.zeros((mc.BATCH_SIZE, mc.ANCHORS)), trainable=False,
      name='iou', dtype=tf.float32
    )
    #QUEUE > 위에 설정해놓은 플레이스홀더 순서대로 큐에 넣어줄 준비 
    self.FIFOQueue = tf.FIFOQueue(
        capacity=mc.QUEUE_CAPACITY,
        dtypes=[tf.float32, tf.float32, tf.float32, 
                tf.float32, tf.float32],
        shapes=[[mc.IMAGE_HEIGHT, mc.IMAGE_WIDTH, 3],
                [mc.ANCHORS, 1],
                [mc.ANCHORS, 4],
                [mc.ANCHORS, 4],
                [mc.ANCHORS, mc.CLASSES]],
    )
    
    #엔큐 오퍼레이션 > 실제 어떤거 넣을지. 
    self.enqueue_op = self.FIFOQueue.enqueue_many(
        [self.ph_image_input, self.ph_input_mask,
         self.ph_box_delta_input, self.ph_box_input, self.ph_labels]
    )
    
    #셔플 배치 안하네? 그냥 배치네? 머 아무튼 그렇다고 치고... Queue로 넣었으니까 뺄때 넣은순서로 리턴해주는듯. 
    self.image_input, self.input_mask, self.box_delta_input, self.box_input, self.labels = tf.train.batch(self.FIFOQueue.dequeue(), 
                                                                                                          batch_size=mc.BATCH_SIZE,
                                                                                                          capacity=mc.QUEUE_CAPACITY) 

    # model parameters # 모델 파라미터 넣을거 
    self.model_params = []

    #이쪽 아래로는 어디서 쓰이는지 잘 모르곘음... 
    # model size counter 
    self.model_size_counter = [] # array of tuple of layer name, parameter size
    # flop counter
    self.flop_counter = [] # array of tuple of layer name, flop number
    # activation counter
    self.activation_counter = [] # array of tuple of layer name, output activations
    self.activation_counter.append(('input', mc.IMAGE_WIDTH*mc.IMAGE_HEIGHT*3))

    ########################### INIT END ###########################
    
    #실제 모델 > 스퀴즈넷으로 설정해둠.
    
    if mc.LOAD_PRETRAINED_MODEL: #일단은 안할꺼지만. 만약 뭐 프리트레인 모델이 있다면 뭐 
        assert tf.gfile.Exists(mc.PRETRAINED_MODEL_PATH), \
            'Cannot find pretrained model at the given path:' \
            '  {}'.format(mc.PRETRAINED_MODEL_PATH)
        self.caffemodel_weight = joblib.load(mc.PRETRAINED_MODEL_PATH)

    #실 모델 
    conv1 = self._conv_layer(
        'conv1', self.image_input, filters=64, size=3, stride=2,
        padding='SAME', freeze=True)
    pool1 = self._pooling_layer(
        'pool1', conv1, size=3, stride=2, padding='SAME')

    fire2 = self._fire_layer(
        'fire2', pool1, s1x1=16, e1x1=64, e3x3=64, freeze=False)
    fire3 = self._fire_layer(
        'fire3', fire2, s1x1=16, e1x1=64, e3x3=64, freeze=False)
    pool3 = self._pooling_layer(
        'pool3', fire3, size=3, stride=2, padding='SAME')

    fire4 = self._fire_layer(
        'fire4', pool3, s1x1=32, e1x1=128, e3x3=128, freeze=False)
    fire5 = self._fire_layer(
        'fire5', fire4, s1x1=32, e1x1=128, e3x3=128, freeze=False)
    pool5 = self._pooling_layer(
        'pool5', fire5, size=3, stride=2, padding='SAME')

    fire6 = self._fire_layer(
        'fire6', pool5, s1x1=48, e1x1=192, e3x3=192, freeze=False)
    fire7 = self._fire_layer(
        'fire7', fire6, s1x1=48, e1x1=192, e3x3=192, freeze=False)
    fire8 = self._fire_layer(
        'fire8', fire7, s1x1=64, e1x1=256, e3x3=256, freeze=False)
    fire9 = self._fire_layer(
        'fire9', fire8, s1x1=64, e1x1=256, e3x3=256, freeze=False)

    # Two extra fire modules that are not trained before
    fire10 = self._fire_layer(
        'fire10', fire9, s1x1=96, e1x1=384, e3x3=384, freeze=False)
    fire11 = self._fire_layer(
        'fire11', fire10, s1x1=96, e1x1=384, e3x3=384, freeze=False)
    dropout11 = tf.nn.dropout(fire11, self.keep_prob, name='drop11')

    num_output = mc.ANCHOR_PER_GRID * (mc.CLASSES + 1 + 4)
    self.preds = self._conv_layer(
        'conv12', dropout11, filters=num_output, size=3, stride=1,
        padding='SAME', xavier=False, relu=False, stddev=0.0001)
    
#RETURN WHAT? 

# 실제 트레인으로.. 모델, CONFIG, DATASET 준비 했음

In [ ]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('dataset', 'KITTI',
                           """Currently only support KITTI dataset.""")
tf.app.flags.DEFINE_string('data_path', '/data/training/image_2', """Root directory of data""")
tf.app.flags.DEFINE_string('image_set', 'train',
                           """ Can be train, trainval, val, or test""")
tf.app.flags.DEFINE_string('year', '2007',
                            """VOC challenge year. 2007 or 2012"""
                            """Only used for Pascal VOC dataset""")
tf.app.flags.DEFINE_string('train_dir', '/squeezeDet',
                            """Directory where to write event logs """
                            """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 1000000,
                            """Maximum number of batches to run.""")
tf.app.flags.DEFINE_string('net', 'squeezeDet',
                           """Neural net architecture. """)
tf.app.flags.DEFINE_string('pretrained_model_path', '',
                           """Path to the pretrained model.""")
tf.app.flags.DEFINE_integer('summary_step', 10,
                            """Number of steps to save summary.""")
tf.app.flags.DEFINE_integer('checkpoint_step', 1000,
                            """Number of steps to save summary.""")
tf.app.flags.DEFINE_string('gpu', '0', """gpu id.""")

#실제 세팅용이라 위랑은 좀 중복될수있음. 

assert FLAGS.dataset == 'KITTI', \
      'Currently only support KITTI dataset'
os.environ['CUDA_VISIBLE_DEVICES'] = FLAGS.gpu


In [ ]:
 with tf.Graph().as_default():
    mc.IS_TRAINING = True
    mc.PRETRAINED_MODEL_PATH = FLAGS.pretrained_model_path #이런거 없지만 뭐 언젠가 필요할 수 있으니... 
   
    imdb = kitti(FLAGS.image_set, FLAGS.data_path, mc) #키티 모양으로 만든다 
    
    
    #로드 데이터 함수 
    def _load_data(load_to_placeholder=True):
        # read batch input
        image_per_batch, label_per_batch, box_delta_per_batch, aidx_per_batch, bbox_per_batch = imdb.read_batch()
        label_indices, bbox_indices, box_delta_values, mask_indices, box_values, = [], [], [], [], []
        
        aidx_set = set()
        num_discarded_labels = 0
        num_labels = 0
        for i in range(len(label_per_batch)): # batch_size
            for j in range(len(label_per_batch[i])): # number of annotations
                num_labels += 1
                if (i, aidx_per_batch[i][j]) not in aidx_set:
                    aidx_set.add((i, aidx_per_batch[i][j]))
                    label_indices.append(
                        [i, aidx_per_batch[i][j], label_per_batch[i][j]])
                    mask_indices.append([i, aidx_per_batch[i][j]])
                    bbox_indices.extend(
                        [[i, aidx_per_batch[i][j], k] for k in range(4)])
                    box_delta_values.extend(box_delta_per_batch[i][j])
                    box_values.extend(bbox_per_batch[i][j])
                else:
                    num_discarded_labels += 1

        if mc.DEBUG_MODE:
            print ('Warning: Discarded {}/({}) labels that are assigned to the same '
                   'anchor'.format(num_discarded_labels, num_labels))

        if load_to_placeholder:
            image_input = model.ph_image_input
            input_mask = model.ph_input_mask
            box_delta_input = model.ph_box_delta_input
            box_input = model.ph_box_input
            labels = model.ph_labels
        else:
            image_input = model.image_input
            input_mask = model.input_mask
            box_delta_input = model.box_delta_input
            box_input = model.box_input
            labels = model.labels

        feed_dict = {
            image_input: image_per_batch,
            input_mask: np.reshape(
              sparse_to_dense(
                  mask_indices, [mc.BATCH_SIZE, mc.ANCHORS],
                  [1.0]*len(mask_indices)),
              [mc.BATCH_SIZE, mc.ANCHORS, 1]),
            box_delta_input: sparse_to_dense(
              bbox_indices, [mc.BATCH_SIZE, mc.ANCHORS, 4],
              box_delta_values),
            box_input: sparse_to_dense(
              bbox_indices, [mc.BATCH_SIZE, mc.ANCHORS, 4],
              box_values),
            labels: sparse_to_dense(
              label_indices,
              [mc.BATCH_SIZE, mc.ANCHORS, mc.CLASSES],
              [1.0]*len(label_indices)),
        }

        return feed_dict, image_per_batch, label_per_batch, bbox_per_batch
    
    def _enqueue(sess, coord):
        try:
            while not coord.should_stop():
                feed_dict, _, _, _ = _load_data()
                sess.run(model.enqueue_op, feed_dict=feed_dict)
                if mc.DEBUG_MODE:
                    print ("added to the queue")
            if mc.DEBUG_MODE:
                print ("Finished enqueue")
        except Exception, e:
            coord.request_stop(e)

######################### 여기까지 데이터 준비 ##################################

    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) #config proto 알아는 보자.. 

    saver = tf.train.Saver(tf.global_variables()) #세이버 그냥 뭐 ㅇㅇ... 
    summary_op = tf.summary.merge_all() #텐서버드 ㅇ

    ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)# 췤포인
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path) #있으면 리스토어

    summary_writer = tf.summary.FileWriter(FLAGS.train_dir, sess.graph) #쓰는거 그냥 텐보

    init = tf.global_variables_initializer() #이니셜
    sess.run(init) #이니셜

    coord = tf.train.Coordinator() #코디 스렛

    if mc.NUM_THREAD > 0: #스레드 여러개 
        enq_threads = [] #배열 맨들고 
        for _ in range(mc.NUM_THREAD): #쓰레드만큼 돌아라
            enq_thread = threading.Thread(target=_enqueue, args=[sess, coord]) #엔큐타겟으로 쓰레등 
            # enq_thread.isDaemon()
            enq_thread.start() #시작하고 
            enq_threads.append(enq_thread) #배열로드가라 

    threads = tf.train.start_queue_runners(coord=coord, sess=sess) #큐너러스탓
    run_options = tf.RunOptions(timeout_in_ms=60000) #런옵션? 이건 처음봄 찾아봐야지 

    # try: 
    for step in xrange(FLAGS.max_steps): #맥스 스텝까지 삥삥 돌면서 
        if coord.should_stop(): #슈드스탑??? 왔이즈디쓰?? 아무튼 이런 컨디션에 걸리브리면  다 멈추고 종료해부러라 
            sess.run(model.FIFOQueue.close(cancel_pending_enqueues=True))
            coord.request_stop()
            coord.join(threads)
            break

        start_time = time.time() #스타트타임 체크 

        if step % FLAGS.summary_step == 0: # 서머리로 저장할 스텝이면  
            feed_dict, image_per_batch, label_per_batch, bbox_per_batch = _load_data(load_to_placeholder=False) #로드데이터 
            op_list = [
                model.train_op, model.loss, summary_op, model.det_boxes,
                model.det_probs, model.det_class, model.conf_loss,
                model.bbox_loss, model.class_loss
            ] #오피 리스트인데... 바꿔야할것 같음. > 안쓰는것도 있을지 모름.. 
            #위에꺼 가지고 아래 스스슥 세션런 ㄱㄱ 
            _, loss_value, summary_str, det_boxes, det_probs, det_class, conf_loss, bbox_loss, class_loss = sess.run(op_list, feed_dict=feed_dict)

            #프레딕션 결과 비쥬얼라이즈 
            _viz_prediction_result(model, image_per_batch, bbox_per_batch, label_per_batch, det_boxes, det_class, det_probs)
            image_per_batch = bgr_to_rgb(image_per_batch)
            #이게 그 이미지인것같음. 
            viz_summary = sess.run(model.viz_op, feed_dict={model.image_to_show: image_per_batch})

            summary_writer.add_summary(summary_str, step) #서머리 해주는 부분 
            summary_writer.add_summary(viz_summary, step)
            summary_writer.flush() #플러시는 꼭 필요한지 모르곘 .. 첨봄 

            print ('conf_loss: {}, bbox_loss: {}, class_loss: {}'.format(conf_loss, bbox_loss, class_loss))
        else: #서머리로 저장할거 아니면... 
            if mc.NUM_THREAD > 0: #스레드 1개넘으면 세션런 돌리고 
                _, loss_value, conf_loss, bbox_loss, class_loss = sess.run(
                  [model.train_op, model.loss, model.conf_loss, model.bbox_loss,
                   model.class_loss], options=run_options)
            else: #스레드 없으면 데이터 로드해와서 세션런 돌리고??? 
                feed_dict, _, _, _ = _load_data(load_to_placeholder=False)
                _, loss_value, conf_loss, bbox_loss, class_loss = sess.run(
                    [model.train_op, model.loss, model.conf_loss, model.bbox_loss,
                    model.class_loss], feed_dict=feed_dict)

        duration = time.time() - start_time #걸린타임 췍

        assert not np.isnan(loss_value), \ 
            'Model diverged. Total loss: {}, conf_loss: {}, bbox_loss: {}, ' \
            'class_loss: {}'.format(loss_value, conf_loss, bbox_loss, class_loss) #로스가 이즈난이 아니면 ??? 다이버즈된다 ㅃㅃ 한다. 

        if step % 10 == 0: #10스텝마다 
            num_images_per_step = mc.BATCH_SIZE#스텝당 배치사이즈만큼
            images_per_sec = num_images_per_step / duration #배치사이즈 나누기 걸린시간 - 이미지당 ? 반대 아닌가?? 
            sec_per_batch = float(duration) #듀레이션 
            format_str = ('%s: step %d, loss = %.2f (%.1f images/sec; %.3f '
                          'sec/batch)')
            print (format_str % (datetime.now(), step, loss_value, images_per_sec, sec_per_batch)) #프린트 
            sys.stdout.flush() #아웃 플러시 해줌. 

      # Save the model checkpoint periodically.
        if step % FLAGS.checkpoint_step == 0 or (step + 1) == FLAGS.max_steps:
            checkpoint_path = os.path.join(FLAGS.train_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=step)